In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
# User Agent
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'}

In [2]:
flats = pd.DataFrame()

In [3]:
# put start page number and end page number
start = 151 # starting page 
end = 200 # end page/
csv_file = f"C:/Users/ROHAN/Desktop/Data Science/campusx/Final Project - real estate/web Scrapping/flats_pune_data-p{start}-{end}.csv"

pageNumber = start 
req = 0
city = 'pune'
    
while pageNumber < end:
    i = 1
    url = f'https://www.99acres.com/flats-in-{city}-ffid-page-{pageNumber}'
    page = requests.get(url, headers = headers)
    # .text provides the content in a decoded Unicode string format, suitable for textual data.
    # .content provides the raw bytes of the response content, suitable for non-textual data.
    pageSoup = BeautifulSoup(page.content, 'html.parser')
    req+=1


    for soup in pageSoup.select_one('div[data-label="SEARCH"]').select('section[data-hydration-on-demand="true"]'):
    # extract property name and property sub-name 
        try:
            property_name = soup.select_one('a.srpTuple__propertyName').text.strip()
            # Extract link
            link = soup.select_one('a.srpTuple__propertyName')['href']
            society = soup.select_one('#srp_tuple_society_heading').text.strip()
        except:
            continue
        # Detail Page
        page = requests.get(link, headers=headers)
        dpageSoup = BeautifulSoup(page.content, 'html.parser')
        req += 1
        try:
            #price Range
            price = dpageSoup.select_one('#pdPrice2').text.strip()
        except:
            price = ''

        # Area
        try:
            area = soup.select_one('#srp_tuple_price_per_unit_area').text.strip()
        except:
            area =''
        # Area with Type
        try:
            areaWithType = dpageSoup.select_one('#factArea').text.strip()
        except:
            areaWithType = ''


        # Configuration
        try:
            bedRoom = dpageSoup.select_one('#bedRoomNum').text.strip()
        except:
            bedRoom = ''
        try:
            bathroom = dpageSoup.select_one('#bathroomNum').text.strip()
        except:
            bathroom = ''
        try:
            balcony = dpageSoup.select_one('#balconyNum').text.strip()
        except:
            balcony = ''

        try:
            additionalRoom = dpageSoup.select_one('#additionalRooms').text.strip()
        except:
            additionalRoom = ''


        # Address

        try:
            address = dpageSoup.select_one('#address').text.strip()
        except:
            address = ''
        # Floor Number
        try:
            floorNum = dpageSoup.select_one('#floorNumLabel').text.strip()
        except:
            floorNum = ''

        try:
            facing = dpageSoup.select_one('#facingLabel').text.strip()
        except:
            facing = ''

        try:
            agePossession = dpageSoup.select_one('#agePossessionLbl').text.strip()
        except:
            agePossession = ''

        # Nearby Locations

        try:
            nearbyLocations = [i.text.strip() for i in dpageSoup.select_one('div.NearByLocation__tagWrap').select('span.NearByLocation__infoText')]
        except:
            nearbyLocations = ''

        # Descriptions
        try:
            description = dpageSoup.select_one('#description').text.strip()
        except:
            description = ''

        # Furnish Details
        try:
            furnishDetails = [i.text.strip() for i in dpageSoup.select_one('#FurnishDetails').select('li')]
        except:
            furnishDetails = ''

        # Features
        if furnishDetails:
            try:
                features = [i.text.strip() for i in dpageSoup.select('#features')[1].select('li')]
            except:
                features = ''
        else:
            try:
                features = [i.text.strip() for i in dpageSoup.select('#features')[0].select('li')]
            except:
                features = ''



        # Rating by Features
        try:
            rating = [i.text for i in dpageSoup.select_one('div.review__rightSide>div>ul>li>div').select('div.ratingByFeature__circleWrap')]
        except:
            rating = ''
        # print(top_f)

        try:
            # Property ID
            property_id = dpageSoup.select_one('#Prop_Id').text.strip()
        except:
            property_id = ''

        # Create a dictionary with the given variables
        property_data = {
        'property_name': property_name,
        'link': link,
        'society': society,
        'price': price,
        'area': area,
        'areaWithType': areaWithType,
        'bedRoom': bedRoom,
        'bathroom': bathroom,
        'balcony': balcony,
        'additionalRoom': additionalRoom,
        'address': address,
        'floorNum': floorNum,
        'facing': facing,
        'agePossession': agePossession,
        'nearbyLocations': nearbyLocations,
        'description': description,
        'furnishDetails': furnishDetails,
        'features': features,
        'rating': rating,
        'property_id': property_id
    }

        
        temp_df = pd.DataFrame.from_records([property_data])
        # print(temp_df)
        flats = pd.concat([flats, temp_df], ignore_index=True)
        i += 1
        if os.path.isfile(csv_file):
        # Append DataFrame to the existing file without header
            temp_df.to_csv(csv_file, mode='a', header=False, index=False)
        else:
            # Write DataFrame to the file with header
            temp_df.to_csv(csv_file, mode='a', header=True, index=False)

        if req % 4==0:
            time.sleep(10)
        if req % 15 == 0:
            time.sleep(50)

    
    print(f'{pageNumber} -> {i}')
    pageNumber += 1


AttributeError: 'NoneType' object has no attribute 'select'